# Pneumonia Autoencoder

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist,cifar10
from tensorflow.keras.models import Model
import keras
from sklearn.manifold import TSNE
import plotly.express as px

os.environ["KAGGLE_USERNAME"] = "ratan123"
os.environ["KAGGLE_KEY"] = "a103a34f897e04ba4ebe09bb64a22132"
!kaggle datasets download -d tolgadincer/labeled-chest-xray-images

In [ ]:
from zipfile import ZipFile
file_name="/content/labeled-chest-xray-images.zip"

with ZipFile(file_name,'r') as zip:
    zip.extractall()
    print('Done')

In [3]:
del zip                                                                         

In [4]:
"""import cv2

lists = ['movietheater', 'bakery', 'subway', 'elevator', 'stairscase', 'prisoncell', 'laundromat']
classes = []
images = []
for dirname, _, filenames in os.walk('/content/indoorCVPR_09/Images/'):
    number_of_images = len(os.listdir(dirname))
    name = dirname.split('/')[-1]
    if number_of_images > 400:
        for file in filenames:
            im = cv2.imread(dirname+'/'+file,0)
            if im is not None:
                images.append(dirname+'/'+file)
                classes.append(name)"""

import cv2
classes = []
images = []
for dirname, _, filenames in os.walk('/content/chest_xray/train/'):
    number_of_images = len(os.listdir(dirname))                                 
    name = dirname.split('/')[-1]
    for file in filenames:
      im = cv2.imread(dirname+'/'+file,0)
      if im is not None:
        images.append(dirname+'/'+file)
        classes.append(name)
        if len(classes) == 1349 and name == "PNEUMONIA":
          break 


In [5]:
classes = classes[:1349*2]
images = images[:1349*2]

In [6]:
import pandas as pd
image_df = pd.DataFrame()
image_df['Filename'] = images
image_df['Label'] = classes

In [7]:
from sklearn.utils import shuffle
image_df = shuffle(image_df)

In [8]:
images_names = image_df['Filename'].to_list()
classes = image_df['Label'].to_list()

In [9]:
images = []
for i in images_names:
  im = cv2.imread(i,0)
  if im is not None:
    images.append(cv2.resize(im,(128,128)))

In [ ]:
plt.imshow(images[0],cmap = 'gray')

In [ ]:
plt.imshow(images[100],cmap = 'gray')

In [12]:
images = np.array(images)

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
classes_transformed = le.fit_transform(classes)

In [14]:
def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 128, 128, 1))
    return array


def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = 0.1
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)


def display(array1, array2):                                                    
    """
    Displays ten random images from each one of the supplied arrays.
    """

    n = 10

    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]

    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1.reshape(128, 128))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2.reshape(128, 128))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()


In [15]:
index = int(images.shape[0]*0.70)

In [ ]:
train_data = images[:index]
test_data = images[index:]

train_classes = classes[:index]
test_classes = classes[index:]

# Normalize and reshape the data
train_data = preprocess(train_data)
test_data = preprocess(test_data)

# Create a copy of the data with added noise
noisy_train_data = noise(train_data)
noisy_test_data = noise(test_data)

# Display the train data and a version of it with added noise
display(train_data, noisy_train_data)

In [ ]:
def rounded_accuracy(y_true,y_pred):
  return keras.metrics.binary_accuracy(tf.round(y_true),tf.round(y_pred))

input = layers.Input(shape=(128, 128, 1))

# Encoder
x = layers.Conv2D(4, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(4, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(4, (3, 3),strides=2, activation="relu", padding="same",name="encoder_output")(x)

# Decoder                                                                       
x = layers.Conv2DTranspose(4, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(4, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(4, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy",metrics = [rounded_accuracy])
autoencoder.summary()

In [ ]:
callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)

import keras
def rounded_accuracy(y_true,y_pred):
  return keras.metrics.binary_accuracy(tf.round(y_true),tf.round(y_pred))

autoencoder.fit(
    x=train_data,
    y=train_data,                                                               
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),callbacks=[callback]                 
)

In [ ]:
predictions = autoencoder.predict(test_data)
display(test_data, predictions)

In [ ]:
callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)

autoencoder.fit(
    x=train_data,
    y=train_data,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),callbacks=[callback]
)

In [ ]:
tf.keras.utils.plot_model(
    autoencoder,
    to_file="autoencoder_ctscans.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)

In [ ]:
predictions = autoencoder.predict(test_data)
display(test_data, predictions)

In [ ]:
#noisy_images = noise(images)

feature_extractor = Model(
    inputs=autoencoder.inputs,
    outputs=autoencoder.get_layer(name="max_pooling2d_1").output,
)

features = feature_extractor(images)
features = features.numpy()

features =features.reshape(features.shape[0],32*32*4)
                                                                                
tsne = TSNE(n_components=2,perplexity = 20)
tsne_results = tsne.fit_transform(features)
fig = px.scatter(tsne_results, x=0, y=1,color=classes,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()

In [24]:
from sklearn.cluster import KMeans                                              
kmeans_tsne = KMeans(n_clusters=2).fit(tsne_results)
kmeans_tsne_labels = kmeans_tsne.labels_

In [ ]:
fig = px.scatter(tsne_results, x=0, y=1,color=kmeans_tsne_labels,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()

In [ ]:
from sklearn import metrics
print(metrics.homogeneity_completeness_v_measure(classes,kmeans_tsne_labels))
print(metrics.rand_score(classes,kmeans_tsne_labels))

In [ ]:
tsne_random = TSNE(n_components=2,init = 'random')
tsne_random_results = tsne_random.fit_transform(features)
fig = px.scatter(tsne_random_results, x=0, y=1,color=classes,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()

In [28]:
kmeans_tsne_random = KMeans(n_clusters=2).fit(tsne_random_results)
kmeans_tsne_random_labels = kmeans_tsne_random.labels_

In [ ]:
fig = px.scatter(tsne_random_results, x=0, y=1,color=kmeans_tsne_random_labels,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()

In [ ]:
from sklearn import metrics
print(metrics.homogeneity_completeness_v_measure(classes,kmeans_tsne_random_labels))
print(metrics.rand_score(classes,kmeans_tsne_random_labels))

In [31]:
kmeans_original = KMeans(n_clusters=2).fit(features)
kmeans_original_labels = kmeans_original.labels_

In [ ]:
print(metrics.homogeneity_completeness_v_measure(classes,kmeans_original_labels)) 
print(metrics.rand_score(classes,kmeans_original_labels))                   

# Self Supervised Learning

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(images[0])

In [22]:
arbitary_labels = []
arbitary_images = []

for image in images:
  arbitary_labels.append(0)
  arbitary_images.append(image)
  arbitary_labels.append(1)
  arbitary_images.append(cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE))
  arbitary_labels.append(2)
  arbitary_images.append(cv2.rotate(image, cv2.ROTATE_180))
  arbitary_labels.append(3)
  arbitary_images.append(cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE))


In [23]:
arbitary_images = np.array(arbitary_images)
arbitary_labels = np.array(arbitary_labels)
arbitary_labels = pd.get_dummies(arbitary_labels).values

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(arbitary_images,arbitary_labels,random_state=1)

In [25]:
model= tf.keras.Sequential()
model.add(layers.Conv2D(kernel_size=(3,3),strides = 2, filters=32, activation='tanh', input_shape=(128,128,1)))
model.add(layers.Conv2D(filters=16,kernel_size = (3,3),strides = 2,activation='tanh'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(filters=16,kernel_size = (3,3),activation='tanh'))

model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu',name = "one")) 
model.add(layers.Dense(128,activation='relu',name = "two"))
model.add(layers.Dense(4,activation = 'softmax'))
    
model.compile(
              loss='categorical_crossentropy', 
              metrics=['acc'],                                                  
              optimizer='adam'
             )

In [ ]:
model.summary()

In [ ]:
model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_val,y_val))

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="selfsupervised_ctscans.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)

In [ ]:
evaluate = model.evaluate(x_val,y_val)
print(evaluate)

In [ ]:
predictions[0].shape

In [ ]:
checkImage = arbitary_images[1000:1001]
checklabel = arbitary_labels[1000:1001]

predict = model.predict(np.array(checkImage))

output = { 0:'No rotation',1:'90 degrees',2:'180 degrees',3:'270 degrees'}

print("Actual :- ",checklabel)
print("Predicted :- ",output[np.argmax(predict)])

In [ ]:
feature_extractor = Model(
    inputs=model.inputs,
    outputs=model.get_layer(name="two").output,
)
features = feature_extractor(images)
features = features.numpy()
print(features.shape)

#features =features.reshape(features.shape[0],32*32*4)
from sklearn.manifold import TSNE
import plotly.express as px
                                                                                
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(features)
fig = px.scatter(tsne_results, x=0, y=1,color=classes,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'}) 
fig.show()

In [60]:
kmeans_tsne = KMeans(n_clusters=2).fit(tsne_results)
kmeans_tsne_labels = kmeans_tsne.labels_

In [ ]:
fig = px.scatter(tsne_results, x=0, y=1,color=kmeans_tsne_labels,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()

In [ ]:
print(metrics.homogeneity_completeness_v_measure(classes,kmeans_tsne_labels))
print(metrics.rand_score(classes,kmeans_tsne_labels))

In [ ]:
tsne_random = TSNE(n_components=2,init = 'random')                              
tsne_random_results = tsne_random.fit_transform(features)
fig = px.scatter(tsne_random_results, x=0, y=1,color=classes,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()

In [49]:
kmeans_tsne_random = KMeans(n_clusters=2).fit(tsne_random_results)
kmeans_tsne_random_labels = kmeans_tsne_random.labels_

In [ ]:
fig = px.scatter(tsne_random_results, x=0, y=1,color=kmeans_tsne_random_labels,labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'})
fig.show()                                                                      

In [ ]:
print(metrics.homogeneity_completeness_v_measure(classes,kmeans_tsne_random_labels))
print(metrics.rand_score(classes,kmeans_tsne_random_labels))

In [52]:
kmeans_original = KMeans(n_clusters=2).fit(features)
kmeans_original_labels = kmeans_original.labels_

In [ ]:
print(metrics.homogeneity_completeness_v_measure(classes,kmeans_original_labels)) 
print(metrics.rand_score(classes,kmeans_original_labels)) 